# Sample Hangul RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


# Load dataset using TextLoader

In [2]:
data_dir    = "data/nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <type 'dict'>, length is 76


data_loader.vocab looks like 
{u'_': 69, u'6': 59, u':': 57, u'\n': 19, u'4': 67, u'5': 63, u'>': 75, u'!': 52, u' ': 1, u'"': 28, u'\u1d25': 0, u"'": 49, u')': 46, u'(': 45, u'-': 65, u',': 27, u'.': 24, u'\u3131': 7, u'0': 73, u'\u3133': 60, u'\u3132': 29, u'\u3135': 50, u'\u3134': 4, u'\u3137': 13, u'\u3136': 44, u'\u3139': 5, u'\u3138': 32, u'\u313b': 55, u'\u313a': 48, u'\u313c': 54, u'?': 41, u'3': 66, u'\u3141': 12, u'\u3140': 51, u'\u3143': 47, u'\u3142': 17, u'\u3145': 10, u'\u3144': 43, u'\u3147': 2, u'\u3146': 22, u'\u3149': 40, u'\u3148': 15, u'\u314b': 42, u'\u314a': 23, u'\u314d': 31, u'\u314c': 30, u'\u314f': 3, u'\u314e': 14, u'\u3151': 34, u'\u3150': 21, u'\u3153': 11, u'\u3152': 74, u'\u3155': 18, u'\u3154': 20, u'\u3157': 9, u'\u3156': 39, u'\u3159': 53, u'\u3158': 26, u'\u315b': 38, u'\u315a': 33, u'\u315d': 36, u'\u315c': 16, u'\u315f': 35, u'\u315e': 61, u'\u3161': 8, u'\u3160': 37, 

# Define Network

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size

with tf.device("/cpu:0"):
    # Select RNN Cell
    unitcell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([unitcell] * num_layers)
    # Set paths to the graph 
    input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    initial_state = cell.zero_state(_batch_size, tf.float32)

    # Set Network
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
            inputs = tf.split(1, _seq_length, tf.nn.embedding_lookup(embedding, input_data))
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
            
    # Loop function for seq2seq
    def loop(prev, _):
        prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
        prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
        return tf.nn.embedding_lookup(embedding, prev_symbol)
    # Output of RNN 
    outputs, last_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state
                                , cell, loop_function=None, scope='rnnlm')
    output = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Next word probability 
    probs = tf.nn.softmax(logits)
    # Define LOSS
    loss = tf.nn.seq2seq.sequence_loss_by_example([logits], # Input
        [tf.reshape(targets, [-1])], # Target
        [tf.ones([_batch_size * _seq_length])], # Weight 
        vocab_size)
    # Define Optimizer
    cost = tf.reduce_sum(loss) / _batch_size / _seq_length
    final_state = last_state
    lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    _optm = tf.train.AdamOptimizer(lr)
    optm = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

Network Ready


In [11]:
# Sample ! 
def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([final_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        # sample = weighted_pick(p)
        # sample = np.argmax(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("sampling function done.")

sampling function done.


# Sample

In [12]:
save_dir = 'data/nine_dreams'
prime = decompose_text(u"누구 ")

print ("Prime Text : %s => %s" % (automata(prime), "".join(prime)))
n = 2000

sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

print (ckpt.model_checkpoint_path)
#print chars
#print vocab
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    sampled_text = sample(sess, chars, vocab, probs, n, prime)
    #print ("")
    print (u"SAMPLED TEXT = %s" % sampled_text)
    print ("")
    print ("-- RESULT --")
    print (automata("".join(sampled_text)))

Prime Text : 누구  => ㄴㅜᴥㄱㅜᴥ 
data/nine_dreams/model.ckpt-99000
SAMPLED TEXT = [u'\u3134', u'\u315c', u'\u1d25', u'\u3131', u'\u315c', u'\u1d25', u' ', u'\u3141', u'\u315c', u'\u3139', u'\u1d25', u'\u3147', u'\u3153', u'\u3145', u'\u1d25', u'\u314e', u'\u314f', u'\u1d25', u'\u3131', u'\u3157', u'\u1d25', u'\n', u'\n', u' ', u' ', u'\u3147', u'\u3153', u'\u1d25', u'\u3134', u'\u3161', u'\u1d25', u' ', u'\u3134', u'\u314f', u'\u3134', u'\u1d25', u'\u3147', u'\u3151', u'\u3147', u'\u1d25', u'\u3131', u'\u3150', u'\u3131', u'\u1d25', u'\u3145', u'\u3163', u'\u1d25', u'\u3131', u'\u314f', u'\u1d25', u' ', u'\u3147', u'\u3163', u'\u1d25', u'\u3141', u'\u3163', u'\u1d25', u' ', u'\u3137', u'\u314f', u'\u1d25', u'\u3148', u'\u314f', u'\u3147', u'\u1d25', u' ', u'\u3134', u'\u3157', u'\u1d25', u'\u3141', u'\u3150', u'\u1d25', u' ', u'\u3132', u'\u315c', u'\u1d25', u'\u3141', u'\u3150', u'\u1d25', u' ', u'\u3147', u'\u314f', u'\u1d25', u'\u3139', u'\u315a', u'\u1d25', u'\u3147', u'\u3163', u'\u1d2